In [1]:
import torch
import numpy as np

In [2]:
# Check The Cuda availability
print(torch.cuda.is_available())
if(torch.cuda.is_available()):
    print(torch.cuda.current_device())
    print(torch.cuda.device(0))
    print(torch.cuda.get_device_name(0))
else: print("Cuda is unavailable")

True
0
NVIDIA GeForce RTX 2080 Ti


## PLY to json

In [3]:
import open3d as o3d
import numpy as np
import copy
meshPath = "./demo_files/scannet_results/000000_pred_confident_nms_bbox.ply"
mesh = o3d.io.read_triangle_mesh(meshPath)

print(mesh.vertices)


std::vector<Eigen::Vector3d> with 280 elements.
Use numpy.asarray() to access data.


In [4]:
print("Cluster connected triangles")
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    triangle_clusters, cluster_n_triangles, cluster_area = (
        mesh.cluster_connected_triangles())
triangle_clusters = np.asarray(triangle_clusters)
cluster_n_triangles = np.asarray(cluster_n_triangles)
cluster_area = np.asarray(cluster_area)
print("Triangle cluster: " + str(np.asarray(triangle_clusters)))
print("Cluster N triangles: " + str(np.asarray(cluster_n_triangles)))
print("Mesh Vertices: " + str(np.asarray(mesh.vertices)))

Cluster connected triangles
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=35
Triangle cluster: [ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1
  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7
  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13
 14 14 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 15 15
 16 16 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17 17 17 17 17
 18 18 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19 19 19 19 19 19 19 19
 20 

In [2]:
print("Show largest cluster")
mesh_1 = copy.deepcopy(mesh)
largest_cluster_idx = 0
triangles_to_remove = triangle_clusters != largest_cluster_idx
mesh_1.remove_triangles_by_mask(triangles_to_remove)
o3d.io.write_triangle_mesh("./demo_files/box.obj", mesh_1)
mesh_1.remove_unreferenced_vertices()
np.asarray(mesh_1.vertices)

Show largest cluster


NameError: name 'copy' is not defined

In [11]:
meshes = []
for i in range (len(np.asarray(cluster_n_triangles))):
    mesh_new = copy.deepcopy(mesh)
    triangles_to_remove = triangle_clusters != i
    mesh_new.remove_triangles_by_mask(triangles_to_remove)
    mesh_new.remove_unreferenced_vertices()
    meshes.append(mesh_new)

print(meshes)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
[TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and

In [24]:
bb = o3d.geometry.OrientedBoundingBox.create_from_points(mesh_1.vertices)
print(bb)
print(np.round(bb.R, 2))

OrientedBoundingBox: center: (-0.370918, -0.0237884, 0.806095), extent: 9.37971, 5.18384, 1.70244)
[[ 0.22 -0.97 -0.05]
 [-0.98 -0.22  0.  ]
 [-0.01  0.05 -1.  ]]


In [32]:
from scipy.spatial.transform import Rotation
rot = Rotation.from_matrix(bb.R).as_euler("xyz", degrees = True)
print(rot)

[177.19267629   0.68850893 -77.31405065]


In [39]:
import json
from scipy.spatial.transform import Rotation

class Coordinates:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def __str__(self):
        return "[" + str(self.x) + "," + str(self.y) + "," + str(self.z) + "]"


class Dimensions:
    def __init__(self, length, width, height):
        self.length = length
        self.width = width
        self.height = height
    def __str__(self):
        return "[" + str(self.length) + "," + str(self.width) + "," + str(self.height) + "]"

class Box:
    name = None
    centroid = None
    dimensions = None
    rotations = None
    
    def __init__(self, name = None, centroid = None, dimensions = None, rotations = None):
        self.name = name
        self.centroid = centroid
        self.dimensions = dimensions
        self.rotations = rotations

    def from_open3d_bb(self, bb: o3d.geometry.OrientedBoundingBox):
        self.centroid = Coordinates(bb.center[0], bb.center[1], bb.center[2])
        self.dimensions = Dimensions(bb.extent[0], bb.extent[1], bb.extent[2])
        rot = Rotation.from_matrix(bb.R).as_euler("xyz", degrees = True)
        self.rotations = Coordinates(rot[0], rot[1], rot[2])

        return self

    def __str__(self):
        return "Bounding Box with: \ncenter: " + str(self.centroid) + "\ndimensions: " + str(self.dimensions) + "\nrotations: " + str(self.rotations)

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__)

class SaveData:
    folder = None
    filename = None
    path = None
    objects = None

    def __init__(self):
       self.folder = None
       self.filename = None
       self.path = None
       self.objects = None

    def from_bounding_boxes(self, bbs):
        self.objects = []
        for bb in bbs:
            self.objects.append(Box().from_open3d_bb(bb))
        return self

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__)




In [ ]:
bbox = Box().from_open3d_bb(bb)
print(bbox)
print(bbox.to_json())

In [40]:
jsonThing = SaveData().from_bounding_boxes([bb])
print(jsonThing.to_json())

{"objects": [{"name": null, "centroid": {"x": -0.3709179585657115, "y": -0.023788375642594537, "z": 0.8060953479045544}, "dimensions": {"length": 9.379706892306311, "width": 5.183838045042352, "height": 1.7024429161942611}, "rotations": {"x": 177.19267629465978, "y": 0.6885089332222982, "z": -77.31405064816396}}]}


In [1]:
import api
meshPath = "./demo_files/scannet_results/000000_pred_confident_nms_bbox.ply"
jsonThing = api.get_json_bounding_boxes(meshPath)

with open("./demo_files/scannet_results/boxes.json", "w") as outfile:
    outfile.write(jsonThing)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=35


/home/jvermandere/.conda/envs/votenet/lib/python3.6/site-packages/scipy/spatial/transform/rotation.py:133: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


In [1]:
path = "/home/jvermandere/projects/votenet/data/Scan1.ply"

import votenetrunner
from utils.pc_util import random_sampling, read_ply
from models.ap_helper import parse_predictions

votenetrunner.calculate(path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2023-03-02 15:48:44,367 - collision - No FCL -- collision checking will not work
WARNING - 2023-03-02 15:48:44,434 - creation - shapely.geometry.Polygon not available!


ModuleNotFoundError: No module named 'votenet'